<a href="https://colab.research.google.com/github/willystumblr/ict-2023-ems/blob/main/%08ict_2023_model_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
from torch import nn, optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


In [3]:
import glob
import tqdm
import os
from collections import defaultdict
from datetime import datetime

In [50]:
PATH = "/content/drive/MyDrive/ict-2023/train"
buildings = defaultdict(pd.DataFrame)
def parser(x):
    return datetime.strptime(x, '%Y %m %d %H')

for path in glob.glob(os.path.join(PATH, "*-revised.csv")):
    building = path.split("/")[-1][:-12]

    data = pd.read_csv(path, index_col=0, parse_dates=[['YEAR', 'MONTH', 'DAY', 'HOUR']], date_parser=parser)
    # dataset = read_csv('raw.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
    t = pd.DataFrame(data.index)
    t_h = pd.DataFrame(data=list(t['YEAR_MONTH_DAY_HOUR'].dt.hour), index=data.index, columns=['Hour'])
    t_m = pd.DataFrame(data=list(t['YEAR_MONTH_DAY_HOUR'].dt.month), index=data.index, columns=['Month'])
    data = pd.concat([data, t_h, t_m], axis=1)
    buildings[building] = data

buildings[building].describe()

,시간당발전량(kWh),수평면(w/㎡),경사면(w/㎡),모듈온도(℃),기온(°C),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),시정(10m),Hour,Month
count,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000,16104.000000
mean,15.895088,159.661575,173.787568,16.335035,14.574044,70.622299,14.250161,8.449447,1008.449242,0.260830,0.641424,5.472740,1614.794647,11.500000,6.268256
std,24.890856,252.452766,275.218672,15.318990,9.988774,22.871057,10.153047,11.750603,8.117557,0.402321,0.932200,3.848155,550.516005,6.922401,3.468447
min,0.000000,0.000000,0.000000,-19.300000,-13.400000,8.000000,0.600000,-28.800000,988.900000,0.000000,0.000000,0.000000,10.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,3.700000,6.400000,54.000000,5.600000,-1.200000,1001.700000,0.000000,0.000000,1.000000,1300.000000,5.750000,3.000000
50%,0.000000,0.000000,0.000000,16.000000,15.100000,72.000000,10.400000,7.500000,1009.200000,0.000000,0.080000,7.000000,1956.000000,11.500000,6.000000
75%,24.000000,245.000000,259.250000,25.700000,23.300000,93.000000,23.125000,19.900000,1014.900000,0.600000,1.080000,9.000000,2000.000000,17.250000,9.000000
max,110.800000,1300.000000,1263.000000,63.300000,34.500000,99.000000,41.900000,29.800000,1027.500000,1.000000,3.880000,10.000000,12000.000000,23.000000,12.000000


# Data preprocessing

We first check the reformatted dataset combined with meteorological data.

In [ ]:
data.head()

,시간당발전량(kWh),수평면(w/㎡),경사면(w/㎡),모듈온도(℃),기온(°C),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),시정(10m),Hour,Month
YEAR_MONTH_DAY_HOUR,,,,,,,,,,,,,,,
2020-06-01 00:00:00,0.0,0.0,0.0,19.0,19.5,89.0,20.1,17.6,1002.9,0.0,0.0,10.0,1652.0,0,6
2020-06-01 01:00:00,0.0,0.0,0.0,18.9,19.3,93.0,20.8,18.1,1003.1,0.0,0.0,10.0,1395.0,1,6
2020-06-01 02:00:00,0.0,0.0,0.0,18.7,19.2,94.0,20.9,18.2,1002.8,0.0,0.0,10.0,1162.0,2,6
2020-06-01 03:00:00,0.0,0.0,0.0,18.7,19.1,96.0,21.2,18.4,1002.7,0.0,0.0,10.0,747.0,3,6
2020-06-01 04:00:00,0.0,0.0,0.0,17.5,18.6,98.0,20.9,18.2,1002.6,0.0,0.0,10.0,509.0,4,6


In [ ]:
data.shape

(19584, 15)

We apply different encoding strategies to features according to their types; MinMaxScaler() for numerical; LabelEncoder() for categorical; and cyclic encoding for cyclic features.

In [51]:
# Separate numerical and categorical features
numerical_features = ['시간당발전량(kWh)', '수평면(w/㎡)', '경사면(w/㎡)', '모듈온도(℃)', '기온(°C)', '습도(%)',
                           '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '일조(hr)',
                           '일사(MJ/m2)', '시정(10m)']
categorical_features = ['전운량(10분위)']
cyclic_features = ['Hour', 'Month']
scaler = MinMaxScaler()


preprocessor = ColumnTransformer(
    transformers=[
        ('num', scaler, numerical_features),
        ('cyc', 'passthrough', [f + '_sin' for f in cyclic_features] + [f + '_cos' for f in cyclic_features]),
        ('cat', 'passthrough', categorical_features)
    ])

In [16]:
# Perform cyclical encoding for cyclic features
def cyclical_encode(df, feature, max_val):
    df[feature + '_sin'] = np.sin(2 * np.pi * df[feature] / max_val)
    df[feature + '_cos'] = np.cos(2 * np.pi * df[feature] / max_val)
    df = df.drop(feature, axis=1)
    return df

In [23]:
np.array(data['시간당발전량(kWh)'])

array([0., 0., 0., ..., 0., 0., 0.])

In [52]:
# Separate input features and target variable
X_dict = dict()
y_dict = dict()
for b, data in buildings.items():
    # Apply cyclical encoding for each cyclic feature
    for feature in cyclic_features:
        max_val = data[feature].max()
        buildings[b] = cyclical_encode(data, feature, max_val)

    X = data[numerical_features + categorical_features + [f + '_sin' for f in cyclic_features] + [f + '_cos' for f in cyclic_features]]
    y = data['시간당발전량(kWh)']

    # Create the ColumnTransformer for preprocessing
    # One-hot encoding for categorical features
    # Keep the numerical and cyclical features as they are
    label_encoder = LabelEncoder()
    X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
    # Apply the preprocessing to the input features
    X_preprocessed = preprocessor.fit_transform(X)
    y_preprocessed = scaler.fit_transform(y.values.reshape(-1, 1))

    # Print the preprocessed shape
    print("Preprocessed shape:", X_preprocessed.shape)

    # Convert the preprocessed data into a PyTorch tensor
    X_tensor = torch.tensor(X_preprocessed, dtype=torch.float32)
    y_tensor = torch.tensor(y_preprocessed, dtype=torch.float32)

    X_dict[b] = X_tensor
    y_dict[b] = y_tensor

<ipython-input-52-3cda416f61bd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
<ipython-input-52-3cda416f61bd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
<ipython-input-52-3cda416f61bd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (16128, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (19584, 17)
Preprocessed shape: (16104, 17)


<ipython-input-52-3cda416f61bd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
<ipython-input-52-3cda416f61bd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['전운량(10분위)'] = label_encoder.fit_transform(data.loc[:, '전운량(10분위)'])
<ipython-input-52-3cda416f61bd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

Shape checking

In [26]:
X_tensor.shape

torch.Size([19584, 17])

In [25]:
y_tensor.shape

torch.Size([19584, 1])

In [53]:
y_dict.keys()

dict_keys(['축구장', '신재생에너지동', '학사과정', '시설관리동', '대학C동', '다산빌딩', '삼성환경동', '중앙도서관', '기숙사B동', '동물실험동', '중앙창고', 'LG도서관', '학생회관', '산업협력관'])

In [29]:
y_dict['산업협력관'].shape

torch.Size([18672, 1])

Since we are using LSTM for the prediction, we create sequences for training. We use the past 7 days' data to predict the next 24-hour solar power generation.

In [54]:
# Define the sequence length (past week's data)
sequence_length = 24 * 7  # 24 hours * 7 days

seq_dict = defaultdict(list)
lab_dict = defaultdict(list)
for b in buildings.keys():
    X_tensor = X_dict[b]
    y_tensor = y_dict[b]
    # Create sequences and labels
    sequences = []
    labels = []
    for i in range(len(X_tensor) - sequence_length - 24):  # Subtract 24 to account for predicting the next 24 hours
        seq = X_tensor[i:i+sequence_length]
        label = y_tensor[i+sequence_length:i+sequence_length+24]  # Next 24 hours
        sequences.append(seq)
        labels.append(label)

    seq_dict[b] = torch.stack(sequences)
    lab_dict[b] = torch.stack(labels)

Shape checking

In [55]:
seq_dict[b].shape, lab_dict[b].shape

(torch.Size([15912, 168, 17]), torch.Size([15912, 24, 1]))

In [56]:
lab_dict[b].squeeze().shape

torch.Size([15912, 24])

# Build Model

We first define custom PyTorch dataset and build model for this task.

In [57]:
# Define a custom PyTorch dataset
class SolarDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx].squeeze()

In [58]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

Set hyperparameters for the model.

In [59]:
# Set the device for training (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the hyperparameters
batch_size = 64
num_epochs = 200
input_size = 17  # Number of features
hidden_size = 64
num_layers = 5
output_size = 24

Set the criterion as MSELoss() to get RSME error.

In [60]:
PATH = "/content/drive/MyDrive/ict-2023/output"
modelsdict = dict()
optimizers = dict()
schedulers = dict()
# Define the loss function and optimizer
criterion = nn.MSELoss()

for b in buildings.keys():
    modelsdict[b] = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
    optimizers[b] = optim.Adam(modelsdict[b].parameters(), lr=0.001)
    schedulers[b] = optim.lr_scheduler.StepLR(optimizers[b], step_size=10, gamma=0.1)

Now, we split the dataset into train and test set. We store each train-test sets pair to a seperate dictionary for convenience.

In [37]:
train_dict = dict()
val_dict = dict()
test_dict = dict()

for b in buildings.keys():
    # Split the data into train, validate and test sets (8:1:1)
    X_train, X_test, y_train, y_test = train_test_split(seq_dict[b], lab_dict[b], test_size=0.2, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

    # Create instances of the custom dataset for train and test sets
    train_dataset = SolarDataset(X_train, y_train)
    val_dataset = SolarDataset(X_val, y_val)
    test_dataset = SolarDataset(X_test, torch.zeros_like(y_test))  # Use zero-filled labels for the test set

    # Create PyTorch DataLoaders for train and test sets
    train_dict[b] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dict[b] = DataLoader(test_dataset, batch_size=batch_size)
    val_dict[b] = DataLoader(val_dataset, batch_size=batch_size)


In [61]:
import time

In [62]:
for b in buildings.keys():
    print(f"====== Training models for {b}======")

    # dataloaders, model, and optimzer for each builing
    train_dataloader = train_dict[b]
    val_dataloader = val_dict[b]

    model = modelsdict[b]
    optimizer = optimizers[b]
    scheduler = schedulers[b]

    checkpoint_path = os.path.join(PATH, b+'-checkpoint-{}.pt'.format(time.asctime(time.localtime())))
    for epoch in tqdm.tqdm(range(0, num_epochs)):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = torch.sqrt(criterion(outputs, labels)) #RMSE Loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Update the learning rate
        scheduler.step()

        # Print the average loss for the epoch
        train_loss = running_loss / len(train_dataloader)


        model.eval()
        running_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Compute loss
                loss = torch.sqrt(criterion(outputs, labels))

                running_loss += loss.item()
        val_loss = running_loss / len(val_dataloader)

        if epoch%10==0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Train loss: {train_loss} Validation Loss: {val_loss}")


    # Save the model checkpoint
    torch.save(model.state_dict(), checkpoint_path)
    modelsdict[b] = model


====== Training models for 축구장======


  0%|          | 1/200 [00:01<05:57,  1.80s/it]

Epoch [1/200] Train loss: 0.13677703484954168 Validation Loss: 0.10351311487536277


  6%|▌         | 11/200 [00:15<03:45,  1.19s/it]

Epoch [11/200] Train loss: 0.08817629120600076 Validation Loss: 0.08744691576688521


...

 90%|█████████ | 181/200 [03:24<00:21,  1.11s/it]

Epoch [181/200] Train loss: 0.055539664880924926 Validation Loss: 0.056418459605554055


 96%|█████████▌| 191/200 [03:35<00:10,  1.17s/it]

Epoch [191/200] Train loss: 0.055541743689672254 Validation Loss: 0.056418459605554055


100%|██████████| 200/200 [03:45<00:00,  1.13s/it]


Testing the models...

In [63]:
for b in buildings.keys():
    model = modelsdict[b]
    test_dataloader = test_dict[b]

    model.eval()
    test_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = torch.sqrt(criterion(outputs, labels))

            test_loss += loss.item()

    # Print the test loss
    test_loss /= len(test_dataloader)
    print(f"[{b}] Test Loss (RSME): {test_loss}")


[축구장] Test Loss (RSME): 0.21503417866845284
[신재생에너지동] Test Loss (RSME): 0.25634345941005215
[학사과정] Test Loss (RSME): 0.21603274393466212
[시설관리동] Test Loss (RSME): 0.24358502078440883
[대학C동] Test Loss (RSME): 0.2683955027211097
[다산빌딩] Test Loss (RSME): 0.22735114155277128
[삼성환경동] Test Loss (RSME): 0.25139750251846926
[중앙도서관] Test Loss (RSME): 0.2372159602180604
[기숙사B동] Test Loss (RSME): 0.5252456644485737
[동물실험동] Test Loss (RSME): 0.25327025498113326
[중앙창고] Test Loss (RSME): 0.23223337146543688
[LG도서관] Test Loss (RSME): 0.23420259452635242
[학생회관] Test Loss (RSME): 0.2476869023615314
[산업협력관] Test Loss (RSME): 0.5057789734725294


# The Next 24-hour Prediction

In [40]:
CHECKPOINT_PATH = "/content/drive/MyDrive/ict-2023/output"

In [41]:
glob.glob(CHECKPOINT_PATH+"/*-checkpoint-*.pt")[0].split("/")[-1].split(".")[0][:-11]

'중앙도서관'

In [42]:
# Load the saved model checkpoint
modelsdict = dict()
for path in glob.glob(CHECKPOINT_PATH+"/*-checkpoint-*.pt"):
    building = path.split("/")[-1].split(".")[0][:-11]
    model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
    model.load_state_dict(torch.load(path))
    modelsdict[building] = model

In [43]:
modelsdict.keys()

dict_keys(['중앙도서관', '시설관리동', '산업협력관', '동물실험동', '신재생에너지동', '기숙사B동', '대학C동', 'LG도서관', '중앙창고', '학생회관', '학사과정', '삼성환경동', '축구장', '다산빌딩'])

The prediction based on the last 7 days' data

In [64]:
pred_PATH = "/content/drive/MyDrive/ict-2023/pred"
data_all = dict() # dataframe for display
for b in modelsdict.keys():
    valid_sequences = []
    valid_sequences.append(X_dict[b][-24*7:]) # predict based on the last 7 days' data
    valid_sequences = torch.stack(valid_sequences)
    valid_sequences.shape

    model = modelsdict[b]
    model.eval()
    prediction = model(valid_sequences.to(device))

    # Convert the predictions tensor to a numpy array
    predictions_array = prediction.detach().cpu().numpy()
    prediction_array = scaler.inverse_transform(predictions_array)

    #convert negative value to positive
    rounded_data = np.maximum(prediction_array, 0.0).reshape(-1,1)

    data_all[b] = rounded_data.flatten()

    # Create a DataFrame from the predictions array
    predictions_df = pd.DataFrame(rounded_data, columns=['Prediction'])
    predictions_df.to_csv(os.path.join(pred_PATH, b+"-revised.csv"), index=False)

In [65]:
pd.DataFrame(data_all)

,축구장,신재생에너지동,학사과정,시설관리동,대학C동,다산빌딩,삼성환경동,중앙도서관,기숙사B동,동물실험동,중앙창고,LG도서관,학생회관,산업협력관
0,1.866336,1.126437,1.392280,1.573860,0.000000,1.515918,0.145057,0.000000,51.279919,0.000000,0.000000,0.330292,1.020343,47.790005
1,2.364634,0.479336,1.708668,0.721285,0.269103,1.946354,0.000000,0.000000,51.358219,0.000000,0.140841,0.282580,1.507836,47.929459
2,1.971907,0.000000,1.334108,1.064272,0.000000,1.630909,0.037323,0.000000,51.482353,0.000000,0.000000,0.987818,0.685207,47.848301
3,1.733098,0.000000,1.270230,0.653245,0.509792,1.486737,0.000000,0.716702,50.988297,0.000000,0.296779,0.523401,0.613219,47.647713
4,1.789875,0.000000,0.772421,0.503603,0.285724,0.968182,0.000000,0.000000,50.939499,0.000000,0.000000,0.000000,1.681623,47.808159
5,2.211731,0.000000,0.000000,0.433303,0.000000,1.345538,0.000000,0.000000,51.001965,0.000000,0.000000,0.000000,0.781920,47.748642
6,2.723249,2.252836,0.994589,1.464828,1.061280,3.978279,0.380708,0.316420,50.803463,0.815486,1.294345,0.556001,2.344135,47.778027
7,5.460021,9.633336,4.997032,3.398059,7.362645,7.431307,3.013373,5.635923,50.946182,5.320480,7.396002,3.956720,8.623003,47.940121
8,10.298738,24.936928,13.791896,6.410751,18.416862,15.322806,7.630166,14.040078,50.714722,14.350310,18.279020,11.836153,20.585133,47.745602
9,15.614820,42.472511,26.080107,12.230748,32.268112,25.488056,13.579660,26.324339,50.881401,25.235195,32.734100,23.707222,35.224934,47.906738


In [ ]:
all = pd.DataFrame(data_all)
all.to_csv("/content/drive/MyDrive/ict-2023/pred/merged.csv", index=False)